# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

ModuleNotFoundError: No module named 'gluonts'

In [2]:
!pip install gluonts

    100% |████████████████████████████████| 296kB 47.8MB/s ta 0:00:01
    100% |████████████████████████████████| 10.4MB 5.2MB/s eta 0:00:01
    100% |████████████████████████████████| 4.8MB 8.5MB/s eta 0:00:01    82% |██████████████████████████▍     | 3.9MB 58.1MB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 34.9MB/s ta 0:00:01
    100% |████████████████████████████████| 28.4MB 1.9MB/s eta 0:00:011
    100% |████████████████████████████████| 81kB 48.2MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 34.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
Successfully built ujson holidays
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [4]:
def deepar(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
#         context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [12]:
%%time
res = deepar(data="m4_daily", seed=42, epochs=200, batches=200)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 23.21it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 8.619 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847150
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.48it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[1] Elapsed time 8.898 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.389316
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.40it/s, avg_epoch_loss=6.23]
INFO:root:Epoch[2] Elapsed time 8.929 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.228093
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.68it/s, avg_epoch_loss=6.14]
INFO:root:Epoch[3] Elapsed time 8.819 s

100%|██████████| 200/200 [00:08<00:00, 22.43it/s, avg_epoch_loss=5.47]
INFO:root:Epoch[36] Elapsed time 8.920 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.465295
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.44it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[37] Elapsed time 8.913 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.455479
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.52it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[38] Elapsed time 8.883 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.453810
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 21.57it/s, avg_epoch_loss=5.44]
INFO:root:Epoch[39] Elapsed time 9.276 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.437232
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.42it/s, avg_epoch_loss=5.49]
INFO:root:Epoch[40] Elapsed time 8.923 second

100%|██████████| 200/200 [00:08<00:00, 22.62it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 8.847 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.407260
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.48it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[74] Elapsed time 8.897 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.380144
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.58it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 8.863 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.420938
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.50it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[76] Elapsed time 8.892 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.370660
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.51it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[77] Elapsed time 8.887 second

INFO:root:Epoch[110] Learning rate is 0.0005
100%|██████████| 200/200 [00:08<00:00, 22.47it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[110] Elapsed time 8.903 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=5.338052
INFO:root:Epoch[111] Learning rate is 0.0005
100%|██████████| 200/200 [00:08<00:00, 22.41it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[111] Elapsed time 8.930 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=5.326169
INFO:root:Epoch[112] Learning rate is 0.0005
100%|██████████| 200/200 [00:08<00:00, 22.31it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[112] Elapsed time 8.967 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.360376
INFO:root:Epoch[113] Learning rate is 0.0005
100%|██████████| 200/200 [00:08<00:00, 22.32it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[113] Elapsed time 8.960 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=5.332659
INFO:root:Epoch[114] Learning rate is 0.0005
100%|██████████| 200/200 [00:08<00:00, 22.49it/s, avg_e

INFO:root:Epoch[146] Learning rate is 0.000125
100%|██████████| 200/200 [00:09<00:00, 22.16it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[146] Elapsed time 9.027 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.318229
INFO:root:Loading parameters from best epoch (136)
INFO:root:Epoch[147] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:09<00:00, 22.07it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[147] Elapsed time 9.066 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.345699
INFO:root:Epoch[148] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:09<00:00, 22.08it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[148] Elapsed time 9.060 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=5.330053
INFO:root:Epoch[149] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:08<00:00, 22.28it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[149] Elapsed time 8.978 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=5.366115
INFO:root:Epoch[150] Learning rate is 6.2

INFO:root:Epoch[182] Learning rate is 5e-05
100%|██████████| 200/200 [00:08<00:00, 22.33it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[182] Elapsed time 8.960 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=5.345273
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 200/200 [00:09<00:00, 22.10it/s, avg_epoch_loss=5.3]
INFO:root:Epoch[183] Elapsed time 9.054 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=5.300156
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 200/200 [00:08<00:00, 22.24it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[184] Elapsed time 8.993 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=5.367657
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 200/200 [00:08<00:00, 22.37it/s, avg_epoch_loss=5.3]
INFO:root:Epoch[185] Elapsed time 8.944 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=5.299638
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 200/200 [00:08<00:00, 22.47it/s, avg_epoch_lo

{'MASE': 3.50221234,
 'MSIS': 40.26351047,
 'epochs': 200,
 'sMAPE': 0.03294363,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.02930601,
 'wQuantileLoss[0.9]': 0.01294468}
CPU times: user 56min 13s, sys: 45.9 s, total: 56min 59s
Wall time: 32min 34s


In [15]:
%%time
res = deepar(data="m4_daily", seed=42, epochs=200, batches=400)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.82it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[0] Elapsed time 17.528 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.618233
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.48it/s, avg_epoch_loss=6.18]
INFO:root:Epoch[1] Elapsed time 17.792 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.184324
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.46it/s, avg_epoch_loss=6.07]
INFO:root:Epoch[2] Elapsed time 17.814 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.067274
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.23it/s, avg_epoch_loss=5.97]
INFO:root:Epoch[3] Elapsed time 17.9

100%|██████████| 400/400 [00:17<00:00, 22.47it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[36] Elapsed time 17.806 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.407976
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.33it/s, avg_epoch_loss=5.4] 
INFO:root:Epoch[37] Elapsed time 17.918 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.400541
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 400/400 [00:18<00:00, 22.12it/s, avg_epoch_loss=5.4] 
INFO:root:Epoch[38] Elapsed time 18.088 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.396853
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.43it/s, avg_epoch_loss=5.4] 
INFO:root:Epoch[39] Elapsed time 17.832 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.395470
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 400/400 [00:17<00:00, 22.51it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[40] Elapsed time 17.772 s

INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 400/400 [00:17<00:00, 22.31it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[73] Elapsed time 17.931 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.338202
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 400/400 [00:17<00:00, 22.24it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[74] Elapsed time 17.991 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.353276
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 400/400 [00:17<00:00, 22.42it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[75] Elapsed time 17.843 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.320806
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 400/400 [00:17<00:00, 22.55it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[76] Elapsed time 17.738 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.326725
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 400/400 [00:17<00:00, 22.38it/s, avg_epoch_loss

100%|██████████| 400/400 [00:17<00:00, 22.53it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[109] Elapsed time 17.757 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=5.318824
INFO:root:Epoch[110] Learning rate is 0.00025
100%|██████████| 400/400 [00:17<00:00, 22.53it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[110] Elapsed time 17.759 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=5.314725
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 400/400 [00:17<00:00, 22.37it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[111] Elapsed time 17.881 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=5.314611
INFO:root:Loading parameters from best epoch (101)
INFO:root:Epoch[112] Learning rate is 0.000125
100%|██████████| 400/400 [00:17<00:00, 22.51it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[112] Elapsed time 17.773 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.310103
INFO:root:Epoch[113] Learning rate is 0.000125
100%|██████████| 400/400 [00:17<00:00, 

INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=5.312857
INFO:root:Epoch[145] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.67it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[145] Elapsed time 17.645 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=5.307158
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.40it/s, avg_epoch_loss=5.3]
INFO:root:Epoch[146] Elapsed time 17.859 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.298233
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.52it/s, avg_epoch_loss=5.3] 
INFO:root:Epoch[147] Elapsed time 17.762 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.300505
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 400/400 [00:18<00:00, 21.77it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[148] Elapsed time 18.379 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=5.279945
INFO:root:Epoch[149] Learning rate is 5e

100%|██████████| 400/400 [00:17<00:00, 22.44it/s, avg_epoch_loss=5.3] 
INFO:root:Epoch[181] Elapsed time 17.828 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=5.295476
INFO:root:Epoch[182] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.47it/s, avg_epoch_loss=5.3] 
INFO:root:Epoch[182] Elapsed time 17.807 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=5.300462
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.44it/s, avg_epoch_loss=5.3] 
INFO:root:Epoch[183] Elapsed time 17.827 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=5.295360
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.56it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[184] Elapsed time 17.732 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=5.307568
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 400/400 [00:17<00:00, 22.45it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[185] Elapsed 

{'MASE': 3.60528637,
 'MSIS': 43.40523917,
 'epochs': 200,
 'sMAPE': 0.03333167,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.03071709,
 'wQuantileLoss[0.9]': 0.01359701}
CPU times: user 1h 47min 25s, sys: 1min 23s, total: 1h 48min 48s
Wall time: 1h 2min 19s


In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results

### DeepAR - use_feat_static==False

In [7]:
def deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
#     cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)
